# Run enformer on variants around the TSS 

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
from kipoi_enformer.veff.dataloader import VCFEnformerDL, get_tss_from_genome_annotation
import tensorflow as tf
from kipoi_enformer.veff.utils import Enformer
from pathlib import Path
from kipoi_enformer.logger import logger
import logging
import os
import pyarrow.parquet as pq
import numpy as np

2024-04-03 17:33:48.176897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 17:33:48.176961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 17:33:48.178076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 17:33:48.185174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/data/nasif12/home_if12/tsi/project

## Config

In [2]:
# show logs in notebook
logging.basicConfig()
logger.setLevel(logging.INFO)

In [3]:
# Check if TensorFlow is able to see your GPU
tf.config.list_physical_devices('GPU')

2024-04-03 17:33:53.317689: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2024-04-03 17:33:53.730016: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 17:33:53.730381: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [4]:
os.getcwd()

'/data/nasif12/home_if12/tsi/projects/kipoi_enformer/notebooks'

In [5]:
base = Path("../example_files/chr22")
files = {
        'fasta': base / "seq.chr22.fa",
        'gtf': base / "annot.chr22.gtf",
        'vcf': base / "promoter_variants.chr22.vcf",
        }

## Run enformer and save results

In [6]:
full_dl = VCFEnformerDL(
        fasta_file=files['fasta'],
        gtf_file=files['gtf'],
        vcf_file=files['vcf'],
        variant_downstream_tss=500,
        variant_upstream_tss=500,
        shift=43,
        seq_length=393_216,
        size=None,
        canonical_only=True,
        protein_coding_only=True,
    )
full_len = len(full_dl)

/opt/modules/i12g/anaconda/envs/kipoi-enformer-george/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/modules/i12g/anaconda/envs/kipoi-enformer-george/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
[W::vcf_parse] Contig 'chr22' is not defined in the header. (Quick workaround: index the file with tabix.)
/opt/modules/i12g/anaconda/envs/kipoi-enformer-george/lib/python3.10/site-packages/pyranges/metho

In [7]:
size = 100
dl = VCFEnformerDL(
        fasta_file=files['fasta'],
        gtf_file=files['gtf'],
        vcf_file=files['vcf'],
        variant_downstream_tss=500,
        variant_upstream_tss=500,
        shift=43,
        seq_length=393_216,
        size=size,
        canonical_only=True,
        protein_coding_only=True,
    )
len(dl)

/opt/modules/i12g/anaconda/envs/kipoi-enformer-george/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/opt/modules/i12g/anaconda/envs/kipoi-enformer-george/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
[W::vcf_parse] Contig 'chr22' is not defined in the header. (Quick workaround: index the file with tabix.)
/opt/modules/i12g/anaconda/envs/kipoi-enformer-george/lib/python3.10/site-packages/pyranges/metho

100

In [8]:
output_dir = Path('../output')
output_dir.mkdir(exist_ok=True, parents=True)
filepath = output_dir / 'enformer_test.parquet'

In [9]:
batch_size = 2

In [10]:
# number of batches for full chr with the given upstream and downstream extensions
full_len // batch_size

17680

In [11]:
# for batch_size 2 it takes ~16s per batch
full_len // batch_size * 16 / 60 / 60 / 24

3.274074074074074

In [12]:
enformer = Enformer()
enformer.predict(dl, batch_size=batch_size, filepath=filepath)

2024-04-03 17:34:11.043169: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 17:34:11.043533: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 17:34:11.043785: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

  0%|          | 0/51 [00:00<?, ?it/s]

[W::vcf_parse] Contig 'chr22' is not defined in the header. (Quick workaround: index the file with tabix.)
/opt/modules/i12g/anaconda/envs/kipoi-enformer-george/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
2024-04-03 17:34:27.312730: W tensorflow/core/kernels/gpu_utils.cc:54] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2024-04-03 17:34:27.323241: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


## Read results file

In [13]:
results_table = pq.read_table(output_dir, columns = ['enformer_start', 'enformer_end', 'landmark_pos', 'chr', 'strand',
                                                    'gene_id', 'transcript_id', 'transcript_start', 'transcript_end',
                                                    'variant_start', 'variant_end', 'ref', 'alt'])

In [14]:
assert results_table.shape == (size, 13)
results_table.shape

(100, 13)

In [15]:
results_table.schema

enformer_start: int64
enformer_end: int64
landmark_pos: int64
chr: string
strand: string
gene_id: string
transcript_id: string
transcript_start: int64
transcript_end: int64
variant_start: int64
variant_end: int64
ref: string
alt: string